# Proyecto Final: 
### Reconocimiento de emociones con visión por computadora

### Preparación dataset

#### Preparación

In [17]:
# Importar librerías y establecer las rutas de los archivos

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

train_dir = 'C:/Users/jmino/OneDrive/Escritorio/archive/images/images/train'
validation_dir = 'C:/Users/jmino/OneDrive/Escritorio/archive/images/images/validation'

#### Cargar imagenes

In [18]:
# Definir ImageDataGenerator para la augmentación y normalización de datos en el conjunto de entrenamiento
train_datagen = ImageDataGenerator(
    rescale=1.0/255,         # Escala los valores de los píxeles a un rango de 0 a 1
    rotation_range=40,       # Rango de rotación aleatoria en grados (aumentado a 40)
    width_shift_range=0.3,   # Rango de desplazamiento horizontal aleatorio como una fracción del ancho total (aumentado a 0.3)
    height_shift_range=0.3,  # Rango de desplazamiento vertical aleatorio como una fracción de la altura total (aumentado a 0.3)
    shear_range=0.3,         # Rango de transformación de corte (aumentado a 0.3)
    zoom_range=0.3,          # Rango de zoom aleatorio (aumentado a 0.3)
    horizontal_flip=True,    # Voltea las imágenes horizontalmente de forma aleatoria
    fill_mode='nearest'      # Estrategia de relleno para los nuevos píxeles que se crean al aplicar las transformaciones
)

# Definir ImageDataGenerator para la normalización de datos en el conjunto de validación
validation_datagen = ImageDataGenerator(rescale=1.0/255)  # Solo escala los valores de los píxeles a un rango de 0 a 1

# Cargar y preprocesar el conjunto de entrenamiento
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),  # Ajusta el tamaño de las imágenes a 64x64 píxeles (puede ajustarse según sea necesario)
    batch_size=32,              # Tamaño del lote
    class_mode='categorical',  # Modo de clasificación: las etiquetas son categóricas
    color_mode='grayscale' # Modo de color: imágenes en escala de grises
)

# Cargar y preprocesar el conjunto de validación
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(64, 64),  # Ajusta el tamaño de las imágenes a 64x64 píxeles (puede ajustarse según sea necesario)
    batch_size=32,         # Tamaño del lote
    class_mode='categorical',  # Modo de clasificación: las etiquetas son categóricas
    color_mode='grayscale'  # Modo de color: imágenes en escala de grises
)

# Nota: Las imágenes de entrenamiento y validación tienen aproximadamente 2 KB, por lo que tienen una baja resolución.
# Es importante tener en cuenta que la baja resolución puede afectar la precisión del modelo.


Found 28821 images belonging to 7 classes.
Found 7066 images belonging to 7 classes.


#### Definición de la CNN

In [19]:
# Definir la arquitectura del modelo CNN ajustada para imágenes en blanco y negro
# Las redes neuronales convolucionales (CNN) son particularmente efectivas para el reconocimiento de imágenes debido a su capacidad para captar características espaciales y patrones en los datos de entrada.

model = Sequential([
    # Primera capa convolucional con 32 filtros de tamaño 3x3, función de activación ReLU y una forma de entrada de 64x64x1 (imágenes en blanco y negro)
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)),  # Nota: El número de canales es 1 para escala de grises
    BatchNormalization(), # (Añadido en la segunda prueba)
    # Aplicación de Max Pooling con una ventana de 2x2 para reducir la dimensionalidad y retener las características más importantes
    MaxPooling2D(pool_size=(2, 2)),

    # Segunda capa convolucional con 64 filtros de tamaño 3x3 y función de activación ReLU
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(), # (Añadido en la segunda prueba)
    # Aplicación de Max Pooling con una ventana de 2x2 para reducir aún más la dimensionalidad
    MaxPooling2D(pool_size=(2, 2)),

    # Tercera capa convolucional con 128 filtros de tamaño 3x3 y función de activación ReLU
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(), # (Añadido en la segunda prueba)
    # Aplicación de Max Pooling con una ventana de 2x2 para reducir nuevamente la dimensionalidad
    MaxPooling2D(pool_size=(2, 2)),

    # (Nueva capa añadida en la segunda prueba)
    Conv2D(256, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    # Capa de Flattening para convertir la matriz resultante de la última capa de pooling en un vector
    Flatten(),

    # Capa densa totalmente conectada con 128 neuronas y función de activación ReLU
    Dense(256, activation='relu', kernel_regularizer=l2(0.001)), # Antes: (Dense(256, activation='relu'),)

    # Capa de Dropout con una tasa de 0.5 para prevenir el sobreajuste durante el entrenamiento
    Dropout(0.5),
    
    # Capa de salida con 7 neuronas (una por cada clase) y función de activación softmax para obtener probabilidades de clasificación
    Dense(7, activation='softmax')  # Suponiendo 7 clases: angry, disgust, fear, happy, neutral, sad, surprise
])


#### Entrenamiento

In [21]:
# Compilación

# Compilar el modelo
# La compilación del modelo es un paso necesario antes de entrenarlo. Aquí se especifican el optimizador, 
# la función de pérdida y las métricas que se utilizarán para evaluar el rendimiento del modelo durante el entrenamiento.

# Parámetro optimizer
# Se ha elegido el optimizador Adam (Adaptive Moment Estimation) porque combina las ventajas de los optimizadores 
# AdaGrad y RMSProp. Adam adapta la tasa de aprendizaje para cada parámetro del modelo, lo que puede conducir a una 
# convergencia más rápida y eficiente. Es conocido por funcionar bien en una amplia gama de problemas de aprendizaje profundo.

# Parámetro learning_rate
# La tasa de aprendizaje se ha fijado en 0.001. Este valor es un buen punto de partida para muchos problemas. 
# Un valor más alto podría hacer que el modelo converja demasiado rápido a un mínimo local, mientras que un valor 
# más bajo podría hacer que el entrenamiento sea innecesariamente lento. Ajustar la tasa de aprendizaje es crucial para 
# el rendimiento del modelo.

# Parámetro loss
# La función de pérdida utilizada es 'categorical_crossentropy'. Esta es la elección estándar para problemas de 
# clasificación multiclase, donde las clases son mutuamente excluyentes. En este caso, estamos clasificando imágenes 
# en una de las 7 categorías de expresiones faciales (angry, disgust, fear, happy, neutral, sad, surprise). 
# La 'categorical_crossentropy' mide la diferencia entre la distribución de las clases verdaderas y las predichas por el modelo.

# Parámetro metrics
# La métrica utilizada es 'accuracy'. La precisión es una métrica intuitiva y fácil de interpretar que mide 
# la fracción de predicciones correctas. Es especialmente útil para evaluar el rendimiento de modelos de clasificación.

# Antes: model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


In [28]:
# Entrenar el modelo
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=40  # Ajusta el número de épocas según sea necesario
)

# Guardar el modelo
model.save('Riley_8.h5')


Epoch 1/40
900/900 [==============================] - 91s 101ms/step - loss: 1.1797 - accuracy: 0.5662 - val_loss: 1.1552 - val_accuracy: 0.5875
Epoch 2/40
900/900 [==============================] - 84s 94ms/step - loss: 1.1838 - accuracy: 0.5680 - val_loss: 1.0923 - val_accuracy: 0.6016
Epoch 3/40
900/900 [==============================] - 85s 94ms/step - loss: 1.1808 - accuracy: 0.5688 - val_loss: 1.0568 - val_accuracy: 0.6271
Epoch 4/40
900/900 [==============================] - 87s 97ms/step - loss: 1.1772 - accuracy: 0.5676 - val_loss: 1.1064 - val_accuracy: 0.5938
Epoch 5/40
900/900 [==============================] - 81s 90ms/step - loss: 1.1803 - accuracy: 0.5655 - val_loss: 1.0894 - val_accuracy: 0.6055
Epoch 6/40
900/900 [==============================] - 80s 88ms/step - loss: 1.1779 - accuracy: 0.5675 - val_loss: 1.2574 - val_accuracy: 0.5459
Epoch 7/40
900/900 [==============================] - 79s 88ms/step - loss: 1.1792 - accuracy: 0.5667 - val_loss: 1.1213 - val_accuracy